In [ ]:
import pandas as pd
from fuzzywuzzy import process

# ----------------------------------------------
# Step 1: Load Data
# ----------------------------------------------
# Replace 'data/input.xlsx' with your actual path in your local environment.
INPUT_FILE_PATH = r"data/input.xlsx"
OUTPUT_FILE_PATH = r"data/output.xlsx"
SHEET_NAME = "YOUR_SHEET_NAME"

df = pd.read_excel(INPUT_FILE_PATH, sheet_name=SHEET_NAME)

# ----------------------------------------------
# Step 2: Define Function for Fuzzy Matching
# ----------------------------------------------
def fuzzy_impute_column(row, column_name, existing_values, threshold=80):
    """
    Imputes missing values using fuzzy string matching.
    
    Parameters:
        row (pd.Series): Row of the DataFrame.
        column_name (str): Target column to impute.
        existing_values (list): List of existing non-null values.
        threshold (int): Minimum fuzzy match score.
    
    Returns:
        str: Imputed value or 'No Match' if below threshold.
    """
    if pd.isnull(row[column_name]):
        best_match, score = process.extractOne(row['KEY_COLUMN'], existing_values)
        return best_match if score >= threshold else "No Match"
    
    return row[column_name]

# ----------------------------------------------
# Step 3: Columns to Impute
# ----------------------------------------------
# Replace column names with placeholders
columns_to_impute = ['COL_GROUP', 'COL_TOP_CUSTOMERS', 'COL_GROUP_AZ']
KEY_COLUMN = "KEY_COLUMN"  # replace with your main matching column name

# ----------------------------------------------
# Step 4: Display missing counts
# ----------------------------------------------
for column in columns_to_impute:
    missing_count = df[column].isnull().sum()
    print(f"Missing values in '{column}': {missing_count}")

# ----------------------------------------------
# Step 5: Apply Imputation
# ----------------------------------------------
for column in columns_to_impute:
    if df[column].isnull().sum() > 0:
        existing_values = df[column].dropna().unique().tolist()
        df[column] = df.apply(
            fuzzy_impute_column, 
            axis=1, 
            column_name=column, 
            existing_values=existing_values
        )
        print(f"Updated '{column}' values:")
        print(df[[KEY_COLUMN, column]].head())
    else:
        print(f"No missing values in '{column}'.")

# ----------------------------------------------
# Step 6: Export Updated File
# ----------------------------------------------
df.to_excel(OUTPUT_FILE_PATH, index=False)
print("Data has been successfully exported!")
